In [ ]:
"""
Testing out the full soma extraction

"""

In [1]:
import cgal_Segmentation_Module as csm
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import time
import trimesh
import numpy as np
import datajoint as dj

In [2]:
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")

Connecting celiib@10.28.0.34:3306


In [ ]:
segment_id = 107816118160698192
version = 0
key = dict(segment_id=segment_id, version = version)

# Getting the Entire Mesh Processed: 

In [3]:
#combine all the meshes into one mesh
def add_mesh_piece(main_mesh_vertices,main_mesh_faces,sub_mesh_vertices,sub_mesh_faces):
    """
    Purpose: Takes in a large mesh piece and an array of other meshes and 
    returns a large mesh with all meshes appended
    
    Parameters:
    main_mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates
    main_mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices
    sub_mesh_vertices(list of np.arrays) : list of np arrays with the vertices arrays for all subsegments to be added
    sub_mesh_faces(list of np.arrays) : list of np arrays with the faces arrays for all subsegments to be added
    
    Returns:
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for NEW CONCATENATED MESH
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for NEW CONCATENATED MESH
    
    
    Pseudocode: 
    - Checks: 
    a. Make sure there sub_mesh arrays are greater than 0 and of the same length

    1) Count the number of vertices and faces in the main mesh
    2) Iterate through the submesh vertices and faces. In loop:
    a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
    b. Add the vertices_count and add that to every number in the faces array
    c. Concatenate the submesh faces onto the larger mesh face
    d. Save this new vertices and faces as the main_mesh verts and faces
    e. Print out how many new vertices and faces added
    3) Print out number of segments added, total faces/vertices for new mesh
    4) Return the main mesh vertices and faces
    
    """
    #a. Make sure there sub_mesh arrays are greater than 0 and of the same length
    if len(sub_mesh_vertices) <= 0:
        print("There were no vertices in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) <= 0:
        print("There were no face in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) != len(sub_mesh_vertices):
        raise Exception("The sub_mesh_faces and sub_mesh_vertices length did not match")
        
    
    #1) Count the number of vertices and faces in the main mesh
    n_main_vertices = len(main_mesh_vertices)
    n_main_faces = len(main_mesh_faces)
    
    
    #2) Iterate through the submesh vertices and faces. In loop:
    for i,(sub_verts, sub_faces) in enumerate(zip(sub_mesh_vertices,sub_mesh_faces)):
        #a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
        n_sub_verts = len(sub_verts)
        n_sub_faces = len(sub_faces)
        
        main_mesh_vertices = np.vstack([main_mesh_vertices,sub_verts])

        
        #b. Add the vertices_count of main to every number in the faces array
        sub_faces = sub_faces + n_main_vertices
        
        #c. Concatenate the submesh faces onto the larger mesh face
        main_mesh_faces = np.vstack([main_mesh_faces,sub_faces])
        
        #d. Save this new vertices and faces as the main_mesh verts and faces (DONE)
        
        #e. Print out how many new vertices and faces added
        #print(f"Added subsegment {i} with {n_sub_verts} vertices and {n_sub_faces} faces")
        
        n_main_vertices = len(main_mesh_vertices)
        n_main_faces = len(main_mesh_faces)
    
    #3) Print out number of segments added, total faces/vertices for new mesh  
    print(f"Added {len(sub_mesh_vertices)} subsegements \n  --> final mesh: {len(main_mesh_vertices)} vertices and {len(main_mesh_faces)} faces")
        
    return main_mesh_vertices,main_mesh_faces 

In [6]:
n = (m65.FromNeuromancer & key).fetch1()

In [7]:
print("---Main Neuron ---")
print(n["segment_id"])
print(n["n_vertices"])
print(n["vertices"])
print(n["faces"])

#get all of the segments and their data
lookup_key = dict(segment_id=n["segment_id"],version = n["version"])
subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)

subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])

subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]


---Main Neuron ---
107816118160698192
937779
[[1219061.75     459752.59375  865277.5    ]
 [1218984.5      459845.03125  865190.25   ]
 [1219070.       459819.375    865243.25   ]
 ...
 [1360432.       734720.       787360.     ]
 [1360528.       734784.       787360.     ]
 [1360640.       734832.       787360.     ]]
[[     0      2      1]
 [     0      1      3]
 [     0      3      4]
 ...
 [937729 937765 937778]
 [937729 937778 937758]
 [937712 937711 937767]]


In [8]:
# creating the entire mesh from the main mesh and all of its sub meshes: 
new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
                   main_mesh_faces=n["faces"],
                   sub_mesh_vertices = subsegments_vertices,
                   sub_mesh_faces=subsegments_faces)

Added 46 subsegements 
  --> final mesh: 1252162 vertices and 2390205 faces


# Running the Meshlab and CGAL algorithms to extract the mesh

In [16]:
s = "_orig_inal.off"
indices = [i for i, a in enumerate(s) if a == "_"]
print(indices)
s[:-(len(s)-indices[-1])]

[0, 5]


'_orig'

In [18]:
import trimesh
original_main = trimesh.Trimesh(vertices=n["vertices"],faces=n["faces"])
output_mesh_name = "temp/" + str(n["segment_id"]) + "_original.off"
original_main.export("./" + output_mesh_name)
print("hello")

hello


In [47]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    command_to_run = 'xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command
    #command_to_run = 'meshlabserver ' + script_command
    
    print(command_to_run)
    subprocess_result = subprocess.run(command_to_run,shell=True)
    
    return subprocess_result

In [48]:
import os, contextlib
import pathlib
import subprocess
def meshlab_fix_manifold_path_specific_mls(input_path_and_filename,
                                           output_path_and_filename="",
                                           segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if input_path_and_filename[-4:] == ".off":
        path_and_filename = input_path_and_filename[:-4]
        input_mesh = input_path_and_filename
    else:
        raise Exception("Not passed off file")
    
    
    if output_path_and_filename == "":
        output_mesh = path_and_filename+"_mls.off"
    else:
        output_mesh = output_path_and_filename
    
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    #print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh


In [49]:
import pathlib
# run the meshlab server script
script_name = "poisson_working_meshlab.mls"
meshlab_script_path_and_name = str(pathlib.Path.cwd()) + "/" + script_name
input_path =str(pathlib.Path.cwd()) + "/" +  output_mesh_name

indices = [i for i, a in enumerate(input_path) if a == "_"]
stripped_ending = input_path[:-(len(input_path)-indices[-1])]

output_path = stripped_ending + "_mls.off"
print(meshlab_script_path_and_name)
print(input_path)
print(output_path)
print("Running the mls function")
meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_path,
                                           output_path_and_filename=output_path,
                                           segment_id=n["segment_id"],
                                           meshlab_script=meshlab_script_path_and_name)

/notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/poisson_working_meshlab.mls
/notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/temp/107816118160698192_original.off
/notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/temp/107816118160698192_mls.off
Running the mls function
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/temp/107816118160698192_original.off -o /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/temp/107816118160698192_mls.off -s /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/poisson_working_meshlab.mls


'/notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/temp/107816118160698192_mls.off'

# The CGAL segmentation 

In [53]:
#import the mesh
new_mesh = trimesh.load_mesh(output_path)

In [54]:
mesh_splits = new_mesh.split(only_watertight=True)

len("Total mesh splits = " + str(mesh_splits))
#get the largest mesh
mesh_lengths = np.array([len(split.faces) for split in mesh_splits])

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()
# sns.distplot(mesh_lengths)

largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
largest_mesh = mesh_splits[largest_index][0]


indices = [i for i, a in enumerate(output_path) if a == "_"]
stripped_ending = output_path[:-(len(output_path)-indices[-1])]
largest_mesh_path = stripped_ending + "_largest_piece.off"

largest_mesh.export(largest_mesh_path)
print("done exporting")

done exporting


In [55]:
segment_id = n["segment_id"]
faces = np.array(largest_mesh.faces)
verts = np.array(largest_mesh.vertices)
#run the whole algorithm on the neuron to test
verts_labels, faces_labels = extract_branches_whole_neuron(import_Off_Flag=False,segment_id=segment_id,vertices=verts,
                     triangles=faces,pymeshfix_Flag=False,
                     import_CGAL_Flag=False,
                     return_Only_Labels=True,
                     clusters=3,
                     smoothness=0.2)

1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.00041937828063964844
2) Staring: Generating CGAL segmentation for neuron
Done writing OFF file

Starting CGAL segmentation
Right before cgal segmentation, clusters = 3, smoothness = 0.2
1
Finished CGAL segmentation algorithm: 119.0459213256836
2) Finished: Generating CGAL segmentation for neuron: 125.82241702079773
3) Staring: Generating Graph Structure and Identifying Soma
soma_index = 1
3) Finished: Generating Graph Structure and Identifying Soma: 0.17187952995300293
5) Staring: Finding Apical Index
Soma Index = 1
Soma Connections = [0, 28, 15, 12, 7, 6]
soma_80_percent = 681368.76
Debugging the axon filter
[(0, 679210.8), (28, 690449.6), (15, 684768.2), (12, 706800.5), (7, 685836.4), (6, 692856.0)]
possible_Axons_filter_1 = [0]
possible_Axons_filter_2 = []
possible_Apical = None
5) Finished: Finding Apical Index: 0.03493165969848633
6) Staring: Clas

In [56]:
from collections import Counter
print(Counter(faces_labels))

Counter({6.0: 104613, 18.0: 40991, 5.0: 29555, 53.0: 19384, 22.0: 17449, 35.0: 15550, 28.0: 13250, 48.0: 12918, 49.0: 12676, 51.0: 12511, 38.0: 10064, 33.0: 9670, 36.0: 7854, 50.0: 7632, 25.0: 5831, 10.0: 5653, 34.0: 3588, 59.0: 3392, 57.0: 3047, 27.0: 2641, 52.0: 2522, 54.0: 2045, 60.0: 1036, 55.0: 692, 56.0: 690, 37.0: 323, 58.0: 140, 31.0: 126, 32.0: 125})


In [59]:
soma_faces = np.where(faces_labels == 5.0)[0]
soma_mesh = largest_mesh.submesh([soma_faces],append=True)

TrackedArray([1297160.16772195,  684347.53524855,  864123.49286675])

# Calculating the center of the soma

In [ ]:
soma_center = soma_mesh.vertices.mean(axis=0).astype("float")
soma_center = soma_center/np.array([4,4,40])
print("Poor man's center from just averagin vertices = " + str(soma_center))
print("Trimesh center of mass = " + str(soma_mesh.center_mass))